# 목표: 각 트랙마다 segment된 짧은 데이터들을 추출하고, Huggingface Dataset으로 만들어 업로드하기

In [1]:
import numpy as np
import pandas as pd
import librosa
import soundfile

import os
from tqdm import tqdm
from pathlib import Path

np.random.seed(2024)

In [2]:
from constants import Constants as C  # import constants

# 데이터 생성
파일들을 순회하며, 각 곡마다 정해진 개수(20개)의 샘플들을 만든다.  
      
곡의 맨 앞과 뒤를 각각 10초씩 자르고, 일정한 길이(10초)의 segment를 등간격으로 추출한다. 
      
총 167곡 x 20 = 3340개의 데이터가 만들어진다.

In [3]:
path = Path('music_samples/')

data = []

for artist_path in path.iterdir():
    artist_name = artist_path.name
    print(artist_name)
    for album_path in artist_path.iterdir():
        album_name = album_path.name
        print(album_name)
        for file_path in tqdm(list(album_path.iterdir())):
            *title, extension = file_path.name.split('.')
            title = '.'.join(title)
            array, sampling_rate = librosa.load(file_path, sr=C.SAMPLING_RATE)
            
            # clip head and tail
            array = array[C.SAMPLING_RATE * C.CLIP_SEC_HEAD : -(C.SAMPLING_RATE * C.CLIP_SEC_TAIL)]
            total_length = len(array)
            
            # 일정한 간격으로부터 segment를 추출
            start_indices = np.linspace(0, total_length - C.SEG_LENGTH , C.NUM_SEGS_PER_SONG)
            start_indices = np.floor(start_indices)
            for seg_id, start_idx in enumerate(start_indices):
                start_idx = int(start_idx)
                end_idx = start_idx + C.SEG_LENGTH
                
                seg = array[start_idx: end_idx]
                data.append({
                    'artist': artist_name,
                    'album': album_name,
                    'title': title,
                    'seg_id': seg_id,
                    'array': seg, 
                })
df = pd.DataFrame(data)

Epik High
Pieces, Pt. 1


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:08<00:00,  1.42it/s]


Fall Out Boy
Infinity on High


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.48it/s]


Madeon
Adventure [Deluxe Edition] Disc 1


...

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:08<00:00,  2.07it/s]


The Best of Suede Disc 2


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:09<00:00,  1.84it/s]


브로콜리 너마저
졸업


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:06<00:00,  1.73it/s]


# Training, Validation, Test Split
같은 곡에서 추출된 서로 다른 데이터가 Training set과 test set에 모두 들어가 있다면, 오버피팅이 염려되어서 데이터 별로 무작위 split은 안될 것 같다.   


같은 곡에서 나온 데이터는 같은 split 안에 들어있도록, 데이터 보다는 track-wise하게 split을 하도록 하자.

In [4]:
# artist별로 샘플링
artist2titles = df.groupby('artist')['title'].agg(set)
valset_titles = set()
testset_titles = set()
for artist, titles in artist2titles.items():
    titles = list(titles)
    num_val = max(2, int(C.VAL_RATIO * len(titles)))
    num_test = max(2, int(C.TEST_RATIO * len(titles)))
    
    val_titles, test_titles = titles[:num_val], titles[num_val: num_val + num_test]
    valset_titles |= {(artist, title) for title in val_titles}
    testset_titles |= {(artist, title) for title in test_titles}
    
df['split'] = df[['artist', 'title']].apply(
    lambda row: 
    ('validation' if tuple(row) in valset_titles
     else 'test' if tuple(row) in testset_titles
    else 'train'), axis=1)

In [5]:
df['split'].value_counts()  # 개수 확인

split
train         2380
test           480
validation     480
Name: count, dtype: int64

In [6]:
# 아티스트 별 나눠진 split 확인

df[['artist', 'title', 'split']].drop_duplicates().groupby(['artist', 'split']).agg('count')

title
artist                   split            
Epik High                test            2
                         train           8
                         validation      2
Fall Out Boy             test            2
                         train          10
                         validation      2
Madeon                   test            2
                         train          14
                         validation      2
Mika                     test            2
                         train           5
                         validation      2
Mr.Big                   test            2
                         train          14
                         validation      2
Muse                     test            3
                         train          18
                         validation      3
Rage Against the Machine test            2
                         train           6
                         validation      2
Red Hot Chili Peppers    test            2
                         train          12
                         validation      2
Suede                    test            5
                         train          25
                         validation      5
브로콜리 너마저                 test            2
                         train           7
                         validation      2

### 생성된 데이터를 wav 포맷으로 저장

In [7]:
# 저장할 파일 명을 컬럼으로 만들어준다
def make_file_name(row):
    return f"music_segments/{row['split']}/{row['artist']}/{row['album']}/{row['title']}_seg{row['seg_id']}.wav"

df['file_name'] = df.apply(make_file_name, axis=1)


for _, row in df.iterrows():
    file_name = row['file_name']
    folder_name = '/'.join(file_name.split('/')[:-1])
    os.makedirs(folder_name, exist_ok=True)
    soundfile.write(file_name, row['array'], C.SAMPLING_RATE)

# Huggingface 데이터셋 업로드 

In [8]:
# Huggingface Dataset으로 만들기 위해 metadata.csv 파일 생성
df_metadata = df[['file_name', 'artist', 'album', 'title', 'seg_id']]
df_metadata.to_csv('metadata.csv', index=False)

In [9]:
from huggingface_hub import login, HfApi
from hf_token import MY_HF_TOKEN

login(MY_HF_TOKEN)
api = HfApi()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\user\.cache\huggingface\token
Login successful


In [10]:
api.upload_folder(
folder_path=f"music_segments",
path_in_repo="music_segments",
repo_id="Hoonvolution/hoons_music_data",
repo_type="dataset",
)

Décalcomanie_seg1.wav:   0%|          | 0.00/320k [00:00<?, ?B/s]

Décalcomanie_seg0.wav:   0%|          | 0.00/320k [00:00<?, ?B/s]

Upload 3340 LFS files:   0%|          | 0/3340 [00:00<?, ?it/s]

Décalcomanie_seg12.wav:   0%|          | 0.00/320k [00:00<?, ?B/s]

Décalcomanie_seg11.wav:   0%|          | 0.00/320k [00:00<?, ?B/s]

...

울지마_seg6.wav:   0%|          | 0.00/320k [00:00<?, ?B/s]

울지마_seg7.wav:   0%|          | 0.00/320k [00:00<?, ?B/s]

울지마_seg8.wav:   0%|          | 0.00/320k [00:00<?, ?B/s]

울지마_seg9.wav:   0%|          | 0.00/320k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Hoonvolution/hoons_music_data/commit/4ef5a2ee77831d5daaf686363c9ff99200efb2b5', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4ef5a2ee77831d5daaf686363c9ff99200efb2b5', pr_url=None, pr_revision=None, pr_num=None)